# Optimization Functions

In [ ]:
import numpy as np
import time

def sphere (x) :
    return np.sum (np.square(x), axis=1)

def matyas (xy) :
    x = xy[:,0]
    y = xy[:,1]
    return 0.26*np.sum(np.square(xy), axis=1) - 0.48*x*y

def bulkin (xy) :
    x = xy[:,0]
    y = xy[:,1]
    return 100*np.sqrt(np.abs(\
                             y - 0.01*np.square(x) 
                             ))
    + 0.01*np.abs(x + 10)

def rastrigin (x) :
    A = 10
    return A*x.shape[1] + np.sum(np.square(x) - A*np.cos(2*np.pi*x) , axis=1)

def alpine (x) :
    return -np.prod (np.sqrt(x)*np.sin(x), axis=1)

def schaffer2 (xy) :
    x2 = np.square (xy[:,0])
    y2 = np.square (xy[:,1])
    return 0.5 + (np.square(np.sin(x2 - y2)) - 0.5)/np.square(1 + 0.001*(x2 + y2))

def schaffer4 (xy) :
    x2 = np.square (xy[:,0])
    y2 = np.square (xy[:,1])
    return 0.5 + (np.square(np.cos(np.sin(np.abs(x2 - y2)))) - 0.5)/np.square(1 + 0.001*(x2 + y2))

def schaffer6 (xy) :
    x = xy[:,0]
    y = xy[:,1]
    sumxy = np.square(x) + np.square(y)
    return 0.5 - (np.square(np.sin(np.sqrt(sumxy))) - 0.5)/np.square(1 + 0.001*sumxy)

def griewank (x) :
    return np.sum(np.square(x), axis=1)/4000 - np.prod(np.cos(x/np.sqrt(np.arange(1, x.shape[1]+1, 1))), axis=1) + 1 
    
def rosenbrock (x) :
    return 100*np.sum(np.square(x[:,1:] - np.square(x[:,:-1])), axis=1) + np.sum(np.square(x[:,:-1]-1), axis=1)

def ackley (x) :
    return np.e + 20 - 20*np.exp(-0.2*np.sqrt(0.5*np.sum(np.square(x), axis=1))) - np.exp(0.5*np.cos(2*np.pi*np.sum(x, axis=1)))

def beale (xy) :
    x = xy[:,0]
    y = xy[:,1]
    return np.square(1.5 - x + x*y) + np.square(2.25 - x + x*np.square(y))\
    + np.square(2.625 - x + x*np.power(y, 3))

def goldstein (xy) :
    x = xy[:,0]
    y = xy[:,1]
    return (1+np.square(x+y+1)*(19 - 14*x + 3*np.square(x) - 14*y + 6*x*y + 3*np.square(y)))*\
    (30 + np.square(2*x - 3*y)*(18 - 32*x + 12*np.square(x) + 48*y - 36*x*y + 27*np.square(y)))

def booth (xy) :
    x = xy[:,0]
    y = xy[:,1]
    return np.square(x + 2*y - 7) + np.square (2*x + y - 5)

def eggholder (xy) :
    x = xy[:,0]
    y = xy[:,1]
    return -(\
    (y+47)*np.sin(np.sqrt(np.abs(\
                                x/2 + y + 47 
                                )))\
    + x*np.sin(np.sqrt(np.abs(\
                              x - y - 47
                             )))
    )

def easom (xy) :
    x = xy[:,0]
    y = xy[:,1]
    return -(\
    np.cos(x) * np.cos(y) * np.exp(-(\
                                     np.square(x-np.pi) + np.square(y-np.pi)
                                    ))
    )

def mccormick (xy) :
    x = xy[:,0]
    y = xy[:,1]
    return np.sin(x+y) + np.square(x-y) - 1.5*x + 2.5*y + 1

def styblinski (x) :
    return np.sum(np.power(x,4) - 16*np.square(x) + 5*x, axis=1)/2

def holdertable (xy) :
    x = xy[:,0]
    y = xy[:,1]
    return -np.abs(\
    np.sin(x) * np.cos(y) * np.exp(np.abs(\
                                   1 - np.sqrt(np.square(x) + np.square(y))/np.pi
                                  ))
    )

def crossintray (xy) :
    x = xy[:,0]
    y = xy[:,1]
    return -0.0001*np.power(1+np.abs(\
    np.sin(x)*np.sin(y)*np.exp(np.abs(\
                                     100 - np.sqrt(np.square(x) + np.square(y))/np.pi  
                                     )+1)                        
    ),0.1)

def threehumpcamel (xy) :
    x = xy[:,0]
    y = xy[:,1]
    return 2*np.square(x) - 1.05*np.power(x,4) + np.power(x,6)/6 + x*y + np.square(y)

def himmelblau (xy) :
    x = xy[:,0]
    y = xy[:,1]
    return np.square(np.square(x) + y - 11) + np.square(x + np.square(y) - 7)

def levi (xy) :
    x = xy[:,0]
    y = xy[:,1]
    return np.square(np.sin(3*np.pi*x)) + np.square(x-1)*(1+np.square(np.sin(3*np.pi*y))) + np.square(y-1)*(1+np.square(np.sin(2*np.pi*y)))

# Inverse Parabolic Confined Distribution

In [ ]:
def pso (obj, llim, rlim, Np=25, w=0.7, c1=2, c2=2, alpha=1.2, beta=0.9, iters=1000) :    
    def ipcd (particle, part, velocity) :
        nonlocal llim, rlim
        
        leftvio = part < llim
        rightvio = part > rlim
        leftRight = np.logical_or(part < llim, part > rlim)
        vio = np.sum (leftRight, axis=1).astype(bool)
        viosum = np.sum(vio)

        if viosum == 0 :
            return part, velocity

        leftvio = leftvio[vio]
        rightvio = rightvio[vio]
        partV = part[vio]
        particleV = particles[vio]

        limvec = np.copy(partV)
        limvec[leftvio] = np.tile (llim, (viosum, 1))[leftvio]
        limvec[rightvio] = np.tile (rlim, (viosum, 1))[rightvio]

        diff = partV - particleV
        Xnot = np.sqrt (np.sum (np.square(diff), axis=1, keepdims=True))
        kvec = np.min (np.where (diff == 0, 1, (limvec - particleV)/diff), axis=1, keepdims=True) 

        bvec = particleV + np.where (diff == 0, 0, kvec*diff)
        dvec = np.sqrt (np.sum (np.square(partV - bvec), axis=1, keepdims=True))

        Xpp = dvec*(1 + alpha*np.tan(np.random.rand(viosum).reshape(-1,1)*np.arctan((Xnot - dvec)/(alpha * dvec))))
        boundk = (Xnot - Xpp)/Xnot

        part[vio] = particleV + np.where (diff == 0, 0, boundk*diff)
        velocity[leftRight] *= -1
        return part, velocity
    
    D = len(llim)
    particles = np.array ([l + (r-l)*np.random.rand(Np) for l, r in zip(llim, rlim)]).transpose()
    velocity = np.random.rand(Np, D)
    momentum = np.zeros (shape=particles.shape)
    pbest = particles
    gbest = min (particles , key = lambda x : obj(x.reshape(1,-1))[0])

    for i in range (0, iters) :
        r1 = np.random.rand (Np, D)
        r2 = np.random.rand (Np, D)

        momentum = beta*momentum + (1-beta)*velocity
        velocity = momentum + c1*r1*(pbest - particles) + c2*r2*(gbest - particles)

        particles, velocity = ipcd (particles, particles+velocity, velocity)  
        less = obj(particles) < obj(pbest)
        pbest[less] = particles[less]
        gbest = min (pbest , key = lambda x : obj(x.reshape(1,-1))[0])
    
    return gbest, particles, velocity

## Test

In [ ]:
# llim = np.repeat(-5, 10)
# rlim = np.repeat(5, 10)
llim = np.array ([-15,-3])
rlim = np.array ([-5, 3])
obj = bulkin

t_st = time.time()
opt, xs, vs = pso (obj, llim, rlim, Np=25, iters=1000)
durtn = time.time() - t_st

print ("f" + str(opt) + " = " + str(obj(opt.reshape(1,-1))[0]))
print ("Time taken = " + str(durtn))